In [ ]:
import numpy as np
from torch.utils.data import ConcatDataset
import pandas as pd
from matplotlib import pyplot as plt
from pprint import pformat

from sklearn.model_selection import ParameterGrid
from importlib import reload
from torch.utils.data import random_split

import modules.orbGridSearch as ogs
import modules.markush as md

reload(md)
reload(ogs)


In [ ]:
MD = md.MarkushDataset.from_dir('./data/training')
#train_set, validation_set = random_split(MD, [0.8, 0.2])
train_set, validation_set, test_set = random_split(MD, [0.6, 0.2, 0.2])

#MD_aux = md.MarkushDataset.from_dir('./data/New_Patent_Images')

In [ ]:
outputs = []

"""param_grid = {
    'in_nr_orb_features': [500, 2000],
    'in_nr_patches': [50, 100, 250],
    'in_xgb_nr_estimators': [500, 1500],
    'in_xgb_max_depth': [6, 15]
}"""

param_grid = {
    'in_nr_orb_features': [2000],
    'in_nr_patches': [100],
    'in_xgb_nr_estimators': [1500],
    'in_xgb_max_depth': [6]
}

In [ ]:
for params in ParameterGrid(param_grid):
    outputs.append(ogs.grid_search(MD=MD, train_set=train_set, validation_set=validation_set, **params))
    pd.DataFrame(outputs).to_csv("Orb_GridSearch_Results.csv") # Save validation results

In [ ]:
best_params = max(outputs, key=lambda x: x['accuracy'])

In [ ]:
# Use test set for evaluating on the best set, train on both validation and train
best_results = ogs.grid_search(MD=MD, train_set=ConcatDataset([train_set, validation_set]), validation_set=test_set,
                                in_nr_orb_features=best_params['in_nr_orb_features'],
                                in_nr_patches=best_params['in_nr_patches'],
                                in_xgb_nr_estimators=best_params['in_xgb_nr_estimators'],
                                in_xgb_max_depth=best_params['in_xgb_max_depth'])

with open('best_results2.txt', 'w') as f:
    f.write("Results on validation set:\n")
    f.write(pformat(best_params))
    f.write("\n\n\n")
    f.write("Results on test set:\n")
    f.write(pformat(best_results))